In [1]:
import sys
import os, os.path

sys.path.append(os.path.join(os.getcwd() ,'/modules'))
root_path = "C:/git/Springboard-Public/Capstone Project 2/"
IN_COLAB = 'google.colab' in sys.modules
if IN_COLAB:
    from google.colab import drive
    drive.mount('/content/drive')
    root_path = "/content/drive/My Drive/Capstone Project 2/"

print('Current Working Dir: ', os.getcwd())
print('Root Path: ', root_path)

# We need to set the working directory since we are using relative paths from various locations
if os.getcwd() != root_path:
  os.chdir(root_path)

Current Working Dir:  C:\git\Springboard-Public\Capstone Project 2\notebooks
Root Path:  C:/git/Springboard-Public/Capstone Project 2/


In [2]:
from modules.lib.StandardTraining import *
from modules.models.ResNet import ResNet_GrayScale

from torchsummary import summary

%matplotlib inline

In [3]:
force_cpu = True
device = torch.device('cuda' if ~force_cpu and torch.cuda.is_available() else 'cpu')
# Assume that we are on a CUDA machine, then this should print a CUDA device:
print(f'Working on device={device}')

Working on device=cuda


In [4]:
class Nets:
    
    def __init__(self):
        self.cur_idx = 0
        self.nets = []
        layers = [18] # ResNet Laysers 18, 34, 50, 101, 152
        lrs = [1e-5]
        batch_sizes = [32]
        observation_min_counts = [None, 20, 50]

        combinations = [(l, lr, bs, m) for l in layers for lr in lrs for bs in batch_sizes for m in observation_min_counts] 

        for combination in combinations:
            layers, lr, batch_size, m = combination   
            name = 'ResNet' + str(layers) + f'\n    learning rate:{lr}\n    batch size:{batch_size}\n    min obs:{m}'
            self.nets.append((name, layers, lr, batch_size, m))
            
    def __iter__(self):
        for name, layers, lr, batch_size, m in self.nets:
            net = ResNet_GrayScale(layers=layers)
            net = nn.DataParallel(net)
            net.to(device)
            yield ModelLoop.getConfigObject(name, net, 
                                            learning_rate=lr, 
                                            batch_size=batch_size,
                                            observation_min_count = m)
    
    def __len__(self):
        return len(self.nets)

In [5]:
modelLoop = ModelLoop(number_images=2500, 
                      default_batch_size=32, 
                      default_learning_rate = 1e-4,
                      num_epochs=4,
                      device=device, 
                      nets=Nets())

C:\git\Springboard-Public\Capstone Project 2\modules\lib\ChextXRayImages.py:251: UserWarning: 
Feature Imbalance Detected (train % - val %):
   Lung_Opacity: 2.26%

  self.warnFeatureImbalance(train, value)
C:\git\Springboard-Public\Capstone Project 2\modules\lib\ChextXRayImages.py:251: UserWarning: 
Feature Imbalance Detected (train % - val %):
   Pneumothorax: 3.31%
   Pleural_Effusion: 3.81%

  self.warnFeatureImbalance(train, value)
C:\git\Springboard-Public\Capstone Project 2\modules\lib\ChextXRayImages.py:251: UserWarning: 
Feature Imbalance Detected (train % - val %):
   Cardiomegaly: 2.73%

  self.warnFeatureImbalance(train, value)


In [6]:
modelLoop.train()

▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆

ResNet18
    learning rate:1e-05
    batch size:32
    min obs:None

▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅
Number of Training Images: 1,979
Number of Validation Images: 521
▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃
Epoch [1/4]
▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔
▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔
Epoch Loss: 33.7255         
Time of Completion: 2020-07-25 09:48:21.600833          
Training Time: 0:00:23.853249          
Validation Time: 0:00:03.159555
▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃
Epoch [2/4]
▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔
▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔
Epoch Loss: 22.2605         
Time of Completion: 2020-07-25 09:48:46.839381          
Training Time: 0:00:22.055056          
Validation Time: 0:00:03.182494
▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃
Epoch [3/4]
▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔
▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔

,Target,True Positive Count,Recall,Precision,F1,ROC AUC,Avg Precision
1,Cardiomegaly,242,0.000000,0.000000,0.000000,0.577922,0.154681
4,Edema,485,0.000000,0.000000,0.000000,0.673940,0.373234
5,Consolidation,101,0.000000,0.000000,0.000000,0.557898,0.063104
7,Atelectasis,318,0.000000,0.000000,0.000000,0.564572,0.193054
9,Pleural_Effusion,674,0.117211,0.607692,0.196517,0.683948,0.502934



VALIDATION

▔▔▔▔▔▔▔


,Target,True Positive Count,Recall,Precision,F1,ROC AUC,Avg Precision
1,Cardiomegaly,60,0.000000,0.000000,0.000000,0.600651,0.175772
4,Edema,124,0.000000,0.000000,0.000000,0.700983,0.415708
5,Consolidation,25,0.000000,0.000000,0.000000,0.520726,0.058921
7,Atelectasis,90,0.000000,0.000000,0.000000,0.569013,0.195219
9,Pleural_Effusion,176,0.085227,0.652174,0.150754,0.691798,0.529824








▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆

ResNet18
    learning rate:1e-05
    batch size:32
    min obs:20

▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅
Number of Training Images: 1,993
Number of Validation Images: 507
▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃
Epoch [1/4]
▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔
▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔
Epoch Loss: 31.3648         
Time of Completion: 2020-07-25 09:50:03.118520          
Training Time: 0:00:22.648471          
Validation Time: 0:00:03.084755
▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃
Epoch [2/4]
▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔
▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔
Epoch Loss: 22.1286         
Time of Completion: 2020-07-25 09:50:28.266311          
Training Time: 0:00:21.959312          
Validation Time: 0:00:03.187481
▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃
Epoch [3/4]
▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔
▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔

...

,Target,True Positive Count,Recall,Precision,F1,ROC AUC,Avg Precision
1,Cardiomegaly,63,0.000000,0.000000,0.000000,0.600565,0.185494
4,Edema,116,0.025862,1.000000,0.050420,0.686568,0.395533
5,Consolidation,23,0.000000,0.000000,0.000000,0.482573,0.072486
7,Atelectasis,88,0.000000,0.000000,0.000000,0.536193,0.188913
9,Pleural_Effusion,157,0.350318,0.611111,0.445344,0.722784,0.541046








▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆

ResNet18
    learning rate:1e-05
    batch size:32
    min obs:50

▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅
Number of Training Images: 1,995
Number of Validation Images: 505
▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃
Epoch [1/4]
▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔
▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔
Epoch Loss: 33.3590         
Time of Completion: 2020-07-25 09:51:46.079352          
Training Time: 0:00:23.079321          
Validation Time: 0:00:03.086750
▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃
Epoch [2/4]
▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔
▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔
Epoch Loss: 22.4033         
Time of Completion: 2020-07-25 09:52:11.182263          
Training Time: 0:00:21.979260          
Validation Time: 0:00:03.121656
▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃
Epoch [3/4]
▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔
▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔

,Target,True Positive Count,Recall,Precision,F1,ROC AUC,Avg Precision
1,Cardiomegaly,252,0.000000,0.000000,0.000000,0.620384,0.200205
4,Edema,486,0.002058,0.500000,0.004098,0.691234,0.385272
5,Consolidation,98,0.000000,0.000000,0.000000,0.592364,0.061693
7,Atelectasis,329,0.000000,0.000000,0.000000,0.583286,0.204724
9,Pleural_Effusion,685,0.243796,0.630189,0.351579,0.698947,0.529037



VALIDATION

▔▔▔▔▔▔▔


,Target,True Positive Count,Recall,Precision,F1,ROC AUC,Avg Precision
1,Cardiomegaly,50,0.000000,0.000000,0.000000,0.653890,0.191642
4,Edema,123,0.008130,1.000000,0.016129,0.725322,0.430814
5,Consolidation,28,0.000000,0.000000,0.000000,0.582135,0.068446
7,Atelectasis,79,0.000000,0.000000,0.000000,0.549771,0.198647
9,Pleural_Effusion,165,0.121212,0.487805,0.194175,0.656720,0.451690
